# Notebook for running evaluations

Imports

In [1]:
import sys
sys.path.append("..")
from experiment.evaluations import evaluate_dispatching #, evaluate_llm_only, evaluate_baseline_rag, evaluate_reranker_rag, evaluate_dense_rag, evaluate_dense_reranker_rag
import nest_asyncio
nest_asyncio.apply()
import os
import numpy as np
import pandas as pd
import json

ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
PROJECT_DIR = os.path.join(ROOT_DIR, "master-thesis-project")
EXPERIMENT_DIR = os.path.join(PROJECT_DIR, "experiment")
DATASET_DIR = os.path.join(EXPERIMENT_DIR, "data")

No abbreviations file found for en. Using default abbreviations.
No abbreviations file found for en. Using default abbreviations.
No abbreviations file found for en. Using default abbreviations.
Calculating embeddings: 16it [00:25,  1.58s/it]
No abbreviations file found for en. Using default abbreviations.
Calculating embeddings: 16it [00:27,  1.70s/it]


Run evaluation

In [2]:
dispatching = await evaluate_dispatching()
# llm_only = await evaluate_llm_only()
# baseline_rag = await evaluate_baseline_rag()
# reranker_rag = await evaluate_reranker_rag()
# dense_rag = await evaluate_dense_rag()
# dense_reranker_rag = await evaluate_dense_reranker_rag()

RAG CALLED
rag result: To get a tax card in Denmark, you need to have MitID, a registered Danish address, and a valid identification like an E-tax password or MitID. You can order the tax card online via skat.dk/tastselv or visit a Citizen Service centre ('Borgerservice') to pick it up. If you're a non-Danish employee, you may need to apply through specific procedures outlined on Skat's website (Source: https://skat.dk/en-us/individuals/tax-card).
PROMPT: How do I get a tax card in Denmark?
DISPATCHED TO: Finance Agent
Finance Agent 8.260868787765503
RAG CALLED
rag result: A preliminary income assessment (forskudsopgørelse) in Denmark is an estimate of your expected income and taxes for the upcoming year. It helps determine your tax budget, deductions, and allowances, and is used to calculate your monthly tax payments. You should update it if your personal or financial situation changes to ensure correct tax payments throughout the year (Sources: https://skat.dk/en-us/individuals/incom

The completion for index 0 has been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


rag result: To rent a home in Denmark, follow these steps:
1. Choose your municipality to access relevant information and services (Source: https://lifeindenmark.borger.dk/housing-and-moving/rental-property/renting-a-home).
2. Find available rental properties through online portals like DanmarkBolig.dk, advertisements, or by asking friends and colleagues.
3. Decide whether to rent from a private landlord or a housing association. Housing associations often have waiting lists and lower rents but are harder for foreigners to access.
4. Obtain a standard rental contract ('standardlejekontrakt') from the Ministry of Social Affairs and Housing website.
5. Before signing, check the lease terms carefully, ensure it describes conditions clearly, and verify the rent, deposit (usually 1 month for rooms, 3 months for flats), and utilities not included.
6. Personally inspect the property, report any defects within 14 days, and do not pay deposits without a signed lease.
7. Notify the landlord or a

The completion for index 0 has been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


rag result: The process for purchasing real property in Denmark involves several steps:

1. **Permission Requirement**: Most buyers need permission from the Department of Civil Affairs unless they have domicile in Denmark or have resided there for at least 5 years (Source: https://lifeindenmark.borger.dk/housing-and-moving/buying/purchasing-real-property).

2. **Determine Property Type**: Identify if the property is a permanent or non-permanent dwelling, as rules differ for each.

3. **Application for Permission**: If required, submit an application to the Department of Civil Affairs with necessary documentation, including proof of residence, purchase details, and personal information. Applications can be made online or via forms available on their website, and processing takes up to 4 weeks (Source: https://lifeindenmark.borger.dk/housing-and-moving/buying/purchasing-real-property).

4. **Land Registration**: When registering ownership, the Court of Land Registration (Tinglysningsrett

...

rag result: The process for early retirement pension in Denmark involves several steps:

1. **Eligibility and Seniority**: Ensure you meet age, citizenship, residence, and attachment to the labor market requirements, including sufficient seniority (accrued from age 16 until about age 61).

2. **Application for Entitlement**: Apply for entitlement through Udbetaling Danmark when you are close to reaching the necessary seniority and age, typically from 3 years before the state retirement age.

3. **Maintain Attachment**: Keep current attachment to the labor market via employment or unemployment insurance for at least 9 months immediately before applying.

4. **Application for Payment**: About three months before you want payments to start, Udbetaling Danmark will contact you with an indicative benefit calculation. You then apply for the actual payment, no earlier than two months before the desired start date.

5. **Payment and Adjustments**: Benefits are paid monthly, up to DKK 14,956 (2

The completion for index 0 has been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


rag result: To claim a deduction for transport between home and work in Denmark, you need to:

1. Ensure your travel distance exceeds 12 km (or 24 km round trip) per day (Source: https://skat.dk/en-us/individuals/deductions-and-allowances/deduction-for-transport/deduction-for-transport-between-home-and-work).
2. Keep track of the days you travel to and from work, as deductions apply only for actual travel days.
3. Calculate your deduction based on the distance traveled, using the applicable rates (e.g., DKK 2.23 per km for 2024 for 25-120 km).
4. Claim the deduction in your tax assessment notice (box 51) or preliminary income assessment (field 417) in E-tax.
5. Keep documentation, especially if traveling long distances or crossing toll bridges, and adjust the distance if your route differs from the system's calculation.
6. If applicable, include any additional deductions for travel in peripheral municipalities or small islands, and for ferry or plane expenses (Source: https://skat.dk/e

The completion for index 0 has been truncated before reaching a natural stopping point. Increase the max_tokens parameter to allow for longer completions.


rag result: To claim deduction for travel expenses in Denmark, you must:

1. Ensure your travel exceeds 12 km (24 km round trip) for work-related travel.
2. Calculate your actual travel distance and keep track of the days you travel.
3. Use the deduction rates: DKK 2.23 per km (2024/2025) for 25-120 km, and DKK 1.12 per km over 120 km, with higher rates for peripheral areas and certain bridges.
4. Claim your deduction in your tax assessment notice (box 51) or preliminary assessment (field 417) via E-tax.
5. Keep documentation such as tickets, receipts, or passport stamps, especially for long-distance or ferry/plane travel.
6. If crossing toll bridges like the Great Belt or Øresund, include additional deductions per trip (e.g., DKK 50-110).
7. Adjust for multiple workplaces or travel routes, and report actual travel distances if they differ from the system's calculation (by changing the km in E-tax).

For detailed steps, see the official guide (Source: https://skat.dk/en-us/individuals/

Read evaluation results

In [2]:
# Dispatching
dispatching = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dispatching.csv"))

# RAG
llm_only = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_llm_only.csv"))
baseline_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_baseline_rag.csv"))
reranker_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_reranker_rag.csv"))
dense_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dense_rag.csv"))
dense_reranker_rag = pd.read_csv(os.path.join(DATASET_DIR, "evaluation_dense_reranker_rag.csv"))

Dispatching

In [12]:

# Assuming dispatching DataFrame has columns: 'expected', 'result', 'time'
if "expected" in dispatching.columns and "result" in dispatching.columns and "time" in dispatching.columns:
    dispatching["correct"] = dispatching["expected"] == dispatching["result"]
    accuracy = dispatching["correct"].mean()
    avg_time = dispatching["time"].mean()
    print(f"Dispatching accuracy: {accuracy:.4f}")
    print(f"Average response time: {avg_time:.4f} seconds")
else:
    print("Required columns ('expected', 'result', 'time') not found in dispatching DataFrame.")

# Filter out rows where result is "InputError" and recalculate summary
if "expected" in dispatching.columns and "result" in dispatching.columns and "time" in dispatching.columns:
    filtered_dispatching = dispatching[dispatching["result"] != "InputError"].copy()
    filtered_dispatching["correct"] = filtered_dispatching["expected"] == filtered_dispatching["result"]
    accuracy = filtered_dispatching["correct"].mean()
    avg_time = filtered_dispatching["time"].mean()
    print(f"Dispatching accuracy (excluding InputError): {accuracy:.4f}")
    print(f"Average response time (excluding InputError): {avg_time:.4f} seconds")

    # Group by 'expected' and count correct and incorrect predictions for filtered data
    filtered_summary = filtered_dispatching.groupby('expected').agg(
        total=('expected', 'count'),
        correct=('correct', 'sum'),
        errors=('correct', lambda x: (~x).sum())
    )
    filtered_summary['accuracy'] = filtered_summary['correct'] / filtered_summary['total']
    print(filtered_summary)
else:
    print("Required columns ('expected', 'result', 'time') not found in dispatching DataFrame.")

# Group by 'expected' and count correct and incorrect predictions
dispatching_summary = dispatching.groupby('expected').agg(
    total=('expected', 'count'),
    correct=('correct', 'sum'),
    errors=('correct', lambda x: (~x).sum())
)
dispatching_summary['accuracy'] = dispatching_summary['correct'] / dispatching_summary['total']
dispatching_summary


Dispatching accuracy: 0.8087
Average response time: 7.4747 seconds
Dispatching accuracy (excluding InputError): 0.8532
Average response time (excluding InputError): 7.4747 seconds
                   total  correct  errors  accuracy
expected                                           
Calculation Agent     59       59       0       1.0
Dreamplan Agent       80       32      48       0.4
Finance Agent        188      188       0       1.0


,total,correct,errors,accuracy
expected,,,,
Calculation Agent,59,59,0,1.000000
Dreamplan Agent,80,32,48,0.400000
Finance Agent,206,188,18,0.912621


RAG response times

In [ ]:
jsonl_files = [
    os.path.join(DATASET_DIR, fname)
    for fname in os.listdir(DATASET_DIR)
    if fname.startswith("questions_") and fname.endswith(".jsonl")
]
avg_response_times = {}
for file in jsonl_files:
    response_times = []
    with open(file, "r") as f:
        for line in f:
            data = json.loads(line)
            if "time" in data:
                response_times.append(data["time"])
    if response_times:
        avg_response_times[os.path.basename(file)] = sum(response_times) / len(response_times)
    else:
        avg_response_times[os.path.basename(file)] = None

avg_response_times

{'questions_dense_reranker_rag.jsonl': 2.6997397374180916,
 'questions_llm_only.jsonl': 3.1922133038344893,
 'questions_dense_rag.jsonl': 2.4557241386580237,
 'questions_reranker_rag.jsonl': 3.042295024232957,
 'questions_baseline_rag.jsonl': 2.14312895524849}

LLM-only vs LLM + RAG

In [7]:
methods = {
    "LLM Only": llm_only,
    "Baseline RAG": baseline_rag,
    "Reranker RAG": reranker_rag,
    "Dense RAG": dense_rag,
    "Dense Reranker RAG": dense_reranker_rag,
}

# Compute mean and std for answer_relevancy for all methods,
# and for other columns for RAG methods (Baseline, Reranker, Dense)
columns_to_compare = ["answer_relevancy", "llm_context_precision_without_reference", "faithfulness", "nv_context_relevance"]

# Compare answer_relevancy for all methods
answer_relevancy_means = {method: df["answer_relevancy"].mean() for method, df in methods.items()}
answer_relevancy_df = pd.DataFrame.from_dict(answer_relevancy_means, orient="index", columns=["answer_relevancy_mean"])

# Compare other metrics for RAG methods only
rag_methods = {k: v for k, v in methods.items() if k != "LLM Only"}
other_metrics = ["llm_context_precision_without_reference", "faithfulness", "nv_context_relevance"]
other_metrics_means = {
    method: {metric: df[metric].mean() for metric in other_metrics}
    for method, df in rag_methods.items()
}
other_metrics_df = pd.DataFrame.from_dict(other_metrics_means, orient="index")

# Map method names to their corresponding avg response time file
method_to_file = {
    "LLM Only": "questions_llm_only.jsonl",
    "Baseline RAG": "questions_baseline_rag.jsonl",
    "Reranker RAG": "questions_reranker_rag.jsonl",
    "Dense RAG": "questions_dense_rag.jsonl",
    "Dense Reranker RAG": "questions_dense_reranker_rag.jsonl",
}


# Add avg response time to answer_relevancy_df
answer_relevancy_df["avg_response_time"] = [
    avg_response_times.get(method_to_file[m], None) for m in answer_relevancy_df.index
]

answer_relevancy_df

,answer_relevancy_mean,avg_response_time
LLM Only,0.770185,3.192213
Baseline RAG,0.861880,2.143129
Reranker RAG,0.842157,3.042295
Dense RAG,0.852620,2.455724
Dense Reranker RAG,0.814216,2.699740


RAG methods comparison

In [10]:
highlighted = other_metrics_df.copy()
for col in highlighted.columns:
    max_val = highlighted[col].max()
    highlighted[col] = highlighted[col].apply(
        lambda x: f"**{x:.4f}**" if np.isclose(x, max_val) else f"{x:.4f}"
    )

# Add and highlight answer_relevancy_mean
highlighted["answer_relevancy_mean"] = answer_relevancy_df.loc[highlighted.index, "answer_relevancy_mean"]
max_relevancy = highlighted["answer_relevancy_mean"].max()
highlighted["answer_relevancy_mean"] = highlighted["answer_relevancy_mean"].apply(
    lambda x: f"**{x:.4f}**" if np.isclose(x, max_relevancy) else f"{x:.4f}"
)

# Add and highlight avg_response_time (lowest is best)
highlighted["avg_response_time"] = [
    avg_response_times.get(method_to_file[m], None) for m in highlighted.index
]
min_time = highlighted["avg_response_time"].min()
highlighted["avg_response_time"] = highlighted["avg_response_time"].apply(
    lambda x: f"**{x:.4f}**" if np.isclose(x, min_time) else f"{x:.4f}"
)

highlighted


,llm_context_precision_without_reference,faithfulness,nv_context_relevance,answer_relevancy_mean,avg_response_time
Baseline RAG,0.7603,**0.9379**,0.5170,**0.8619**,**2.1431**
Reranker RAG,0.7410,0.9142,0.5121,0.8422,3.0423
Dense RAG,0.7673,0.9308,**0.6262**,0.8526,2.4557
Dense Reranker RAG,**0.7686**,0.9198,0.6238,0.8142,2.6997
